# $$ Part 01 - Refresh $$
### To check comments visit Part 01

In [ ]:
!pip install patchify 

In [ ]:
import os
import cv2
from PIL import Image 
import numpy as np 
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from matplotlib import pyplot as plt
import random

In [ ]:
minmaxscaler = MinMaxScaler()

In [ ]:
dataset_root_folder = '/content/drive/MyDrive/Colab Notebooks/datasets/satellite/'


In [ ]:
dataset_name = "DubaiDataset"


In [ ]:
image_patch_size = 256


In [ ]:
image_dataset = []
mask_dataset = []

for image_type in ['images' , 'masks']:
  if image_type == 'images':
    image_extension = 'jpg'
  elif image_type == 'masks':
     image_extension = 'png'
  for tile_id in range(1,8):
    for image_id in range(1,20):
      image = cv2.imread(f'{dataset_root_folder}/{dataset_name}/Tile {tile_id}/{image_type}/image_part_00{image_id}.{image_extension}',1)
      if image is not None:
        if image_type == 'masks':
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #print(image.shape)
        size_x = (image.shape[1]//image_patch_size)*image_patch_size
        size_y = (image.shape[0]//image_patch_size)*image_patch_size
        #print("{} --- {} - {}".format(image.shape, size_x, size_y))
        image = Image.fromarray(image)
        image = image.crop((0,0, size_x, size_y))
        #print("({},  {})".format(image.size[0],image.size[1]))
        image = np.array(image)
        patched_images = patchify(image, (image_patch_size, image_patch_size, 3), step=image_patch_size)
        #print(len(patched_images))
        for i in range(patched_images.shape[0]):
          for j in range(patched_images.shape[1]):
            if image_type == 'images':
              individual_patched_image = patched_images[i,j,:,:]
              #print(individual_patched_image.shape)
              individual_patched_image = minmaxscaler.fit_transform(individual_patched_image.reshape(-1, individual_patched_image.shape[-1])).reshape(individual_patched_image.shape)
              individual_patched_image = individual_patched_image[0]
              #print(individual_patched_image.shape)
              image_dataset.append(individual_patched_image)
            elif image_type == 'masks':
              individual_patched_mask = patched_images[i,j,:,:]
              individual_patched_mask = individual_patched_mask[0]
              mask_dataset.append(individual_patched_mask)

In [ ]:

image_dataset = np.array(image_dataset)
mask_dataset = np.array(mask_dataset)


In [ ]:
print(len(image_dataset))
print(len(mask_dataset))

In [ ]:
random_image_id = random.randint(0, len(image_dataset))

plt.figure(figsize=(14,8))
plt.subplot(121)
plt.imshow(image_dataset[random_image_id])
plt.subplot(122)
plt.imshow(mask_dataset[random_image_id])

In [ ]:

class_building = '#3C1098'
class_building = class_building.lstrip('#')
class_building = np.array(tuple(int(class_building[i:i+2], 16) for i in (0,2,4)))
print(class_building)

class_land = '#8429F6'
class_land = class_land.lstrip('#')
class_land = np.array(tuple(int(class_land[i:i+2], 16) for i in (0,2,4)))
print(class_land)

class_road = '#6EC1E4'
class_road = class_road.lstrip('#')
class_road = np.array(tuple(int(class_road[i:i+2], 16) for i in (0,2,4)))
print(class_road)

class_vegetation = '#FEDD3A'
class_vegetation = class_vegetation.lstrip('#')
class_vegetation = np.array(tuple(int(class_vegetation[i:i+2], 16) for i in (0,2,4)))
print(class_vegetation)

class_water = '#E2A929'
class_water = class_water.lstrip('#')
class_water = np.array(tuple(int(class_water[i:i+2], 16) for i in (0,2,4)))
print(class_water)

class_unlabeled = '#9B9B9B'
class_unlabeled = class_unlabeled.lstrip('#')
class_unlabeled = np.array(tuple(int(class_unlabeled[i:i+2], 16) for i in (0,2,4)))
print(class_unlabeled)
     

In [ ]:
label = individual_patched_mask


In [ ]:
def rgb_to_label(label):
  label_segment = np.zeros(label.shape, dtype=np.uint8)
  label_segment[np.all(label == class_water, axis=-1)] = 0
  label_segment[np.all(label == class_land, axis=-1)] = 1
  label_segment[np.all(label == class_road, axis=-1)] = 2
  label_segment[np.all(label == class_building, axis=-1)] = 3
  label_segment[np.all(label == class_vegetation, axis=-1)] = 4
  label_segment[np.all(label == class_unlabeled, axis=-1)] = 5
  #print(label_segment)
  label_segment = label_segment[:,:,0]
  #print(label_segment)
  return label_segment

In [ ]:

labels = []
for i in range(mask_dataset.shape[0]):
  label = rgb_to_label(mask_dataset[i])
  labels.append(label)

In [ ]:
labels = np.array(labels)
labels = np.expand_dims(labels, axis=3)

In [ ]:

np.unique(labels)
     

In [ ]:
print("Total unique labels based on masks: ",format(np.unique(labels)))

In [ ]:
random_image_id = random.randint(0, len(image_dataset))

plt.figure(figsize=(14,8))
plt.subplot(121)
plt.imshow(image_dataset[random_image_id])
plt.subplot(122)
#plt.imshow(mask_dataset[random_image_id])
plt.imshow(labels[random_image_id][:,:,0])

In [ ]:
total_classes = len(np.unique(labels))


In [ ]:
total_classes


In [ ]:
from tensorflow.keras.utils import to_categorical 


In [ ]:

labels_categorical_dataset = to_categorical(labels, num_classes=total_classes)


In [ ]:
master_trianing_dataset = image_dataset


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(master_trianing_dataset, labels_categorical_dataset, test_size=0.15, random_state=100)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:

image_height = X_train.shape[1]
image_width = X_train.shape[2]
image_channels = X_train.shape[3]
total_classes = y_train.shape[3]

In [ ]:

print(image_height)
print(image_width)
print(image_channels)
print(total_classes)

# $$ Part 01 -Starts Here $$

In [ ]:
# Install or upgrade the segmentation-models library, which is used for semantic segmentation tasks.
!pip install -U segmentation-models

In [ ]:
# Import essential modules for building a convolutional neural network (CNN) model.
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.layers import concatenate, BatchNormalization, Dropout, Lambda


In [ ]:
# Import the Keras backend module as 'K'.
from keras import backend as K


In [ ]:
def jaccard_coef(y_true, y_pred):
  # Flatten the true and predicted segmentation masks to 1D arrays.
  y_true_flatten = K.flatten(y_true)
  y_pred_flatten = K.flatten(y_pred)

  # Calculate the intersection between true and predicted mask values.
  intersection = K.sum(y_true_flatten * y_pred_flatten)

  # Compute the Jaccard coefficient, also known as Intersection over Union (IoU).
  final_coef_value = (intersection + 1.0) / (K.sum(y_true_flatten) + K.sum(y_pred_flatten) - intersection + 1.0)

  return final_coef_value


In [ ]:
# Define a U-Net model for image segmentation.
def multi_unet_model(n_classes=5, image_height=256, image_width=256, image_channels=1):
    # Define the input layer for the model.
    inputs = Input((image_height, image_width, image_channels))

    # Create a reference to the input layer for future concatenation.
    source_input = inputs

    # Contracting path (Encoder)
    c1 = Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(source_input)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c1)
    p1 = MaxPooling2D((2,2))(c1)

    c2 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c2)
    p2 = MaxPooling2D((2,2))(c2)

    c3 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c3)
    p3 = MaxPooling2D((2,2))(c3)

    c4 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c4)
    p4 = MaxPooling2D((2,2))(c4)

    c5 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p4)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c5)

    # Expansive path (Decoder)
    u6 = Conv2DTranspose(128, (2,2), strides=(2,2), padding="same")(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c6)

    u7 = Conv2DTranspose(64, (2,2), strides=(2,2), padding="same")(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c7)

    u8 = Conv2DTranspose(32, (2,2), strides=(2,2), padding="same")(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c8)

    u9 = Conv2DTranspose(16, (2,2), strides=(2,2), padding="same")(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c9)

    # Output layer for segmentation masks.
    outputs = Conv2D(n_classes, (1,1), activation="softmax")(c9)

    # Create and return the U-Net model.
    model = Model(inputs=[inputs], outputs=[outputs])
    return model


In [ ]:
# Define a list of evaluation metrics to be used during model training and validation.
# In this case, two metrics are defined: "accuracy" and "jaccard_coef".

# "accuracy" measures the overall accuracy of the model's predictions.
# It's a common metric for classification tasks where the goal is to correctly classify each pixel.

# "jaccard_coef" computes the Jaccard coefficient (intersection over union) for segmentation masks.
# This metric measures the similarity between the predicted and true segmentation masks.
metrics = ["accuracy", jaccard_coef]


In [ ]:
# Print the height of the input images.
print(image_height)

# Print the width of the input images.
print(image_width)

# Print the number of color channels in the input images (e.g., 1 for grayscale, 3 for RGB).
print(image_channels)

# Print the total number of classes for image segmentation.
print(total_classes)


In [ ]:
# This function returns a deep learning model for image segmentation.
def get_deep_learning_model():
  # Calls the 'multi_unet_model' function to create a U-Net model for image segmentation.
  return multi_unet_model(n_classes=total_classes, 
                          image_height=image_height, 
                          image_width=image_width, 
                          image_channels=image_channels)


In [ ]:
# Get the deep learning model using the defined function 'get_deep_learning_model'.
model = get_deep_learning_model()


In [ ]:
# Please uncomment this line to get model confiuration
# model.get_config()

# Generating Loss Function
- dice loss > Focal Loss > Total Loss
- Total Loss = (Dice loss + (1*Focal Loss))

In [ ]:
# Define a list of weights, each representing the relative weight for a specific class.
weights = [0.1666, 0.1666, 0.1666, 0.1666, 0.1666, 0.1666]


In [ ]:
# Initialize a Dice Loss object
# This is a loss function commonly used in image segmentation tasks.
dice_loss = sm.losses.DiceLoss(
    class_weights=weights
)

In [ ]:
# Create a Focal Loss object for multi-class image segmentation.
focal_loss = sm.losses.CategoricalFocalLoss()


In [ ]:
# Import the Segmentation Models library (segmentation_models) as "sm."
import segmentation_models as sm


In [ ]:
# Calculate the total loss as a combination of Dice Loss and Focal Loss.
total_loss = dice_loss + (1 * focal_loss)


# Model compilation

In [ ]:
import tensorflow as tf
     

In [ ]:
# Clear the TensorFlow Keras backend session to release any allocated memory and resources.
tf.keras.backend.clear_session()


In [ ]:
# Compile the deep learning model using the Adam optimizer, total loss function, and specified evaluation metrics.
model.compile(optimizer="adam", loss=total_loss, metrics=metrics)


In [ ]:
# Display a summary of the deep learning model's architecture.
model.summary()


In [ ]:
# Train the deep learning model on the training data.
model_history = model.fit(X_train, y_train,
                          batch_size=16,
                          verbose=1,
                          epochs=100,
                          validation_data=(X_test, y_test),
                          shuffle=False)


In [ ]:
# Extract training and validation loss values from the training history.
loss = history_a.history['loss']
val_loss = history_a.history['val_loss']

# Create a plot of training and validation loss over epochs.
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label="Training Loss")  # Plot training loss in yellow
plt.plot(epochs, val_loss, 'r', label="Validation Loss")  # Plot validation loss in red

# Add labels and a legend to the plot.
plt.title("Training Vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

# Display the plot.
plt.show()


In [ ]:
# Extract training and validation IoU (Jaccard coefficient) values from the training history.
jaccard_coef = history_a.history['jaccard_coef']
val_jaccard_coef = history_a.history['val_jaccard_coef']

# Create a plot of training and validation IoU over epochs.
epochs = range(1, len(jaccard_coef) + 1)
plt.plot(epochs, jaccard_coef, 'y', label="Training IoU")  # Plot training IoU in yellow
plt.plot(epochs, val_jaccard_coef, 'r', label="Validation IoU")  # Plot validation IoU in red

# Add labels and a legend to the plot.
plt.title("Training Vs Validation IoU")
plt.xlabel("Epochs")
plt.ylabel("IoU (Jaccard Coefficient)")
plt.legend()

# Display the plot.
plt.show()


In [ ]:
# Access the model's training parameters
model_history.params


In [ ]:
# Generate predictions on the test data using the trained model
y_pred = model.predict(X_test)


In [ ]:
# Get the number of predictions in the y_pred array
len(y_pred)


In [ ]:
# y_pred contains the predictions made by the deep learning model
y_pred


In [ ]:
# y_pred_argmax contains the class labels with the highest probability for each pixel
y_pred_argmax = np.argmax(y_pred, axis=3)


In [ ]:
# Calculate the length (number of elements) in the y_pred_argmax array
len(y_pred_argmax)


In [ ]:
# Display the y_pred_argmax array
y_pred_argmax


In [ ]:
# Calculate the argmax of y_test to get the true class labels
y_test_argmax = np.argmax(y_test, axis=3)


In [ ]:
# Obtain the true class labels from y_test_argmax
y_test_argmax


# Comparing prediction results
- Using test images using mask images and predicted result images

In [ ]:
import random

In [ ]:
# Select a random test image from the test dataset.
test_image_number = random.randint(0, len(X_test))
test_image = X_test[test_image_number]

# Retrieve the corresponding ground truth (actual) segmentation mask.
ground_truth_image = y_test_argmax[test_image_number]

# Prepare the test image input for prediction by expanding its dimensions.
test_image_input = np.expand_dims(test_image, 0)

# Use the model to predict the segmentation mask for the test image.
prediction = model.predict(test_image_input)

# Extract the predicted segmentation mask and convert it to a 2D format.
predicted_image = np.argmax(prediction, axis=3)
predicted_image = predicted_image[0, :, :]


In [ ]:
# Create a visualization of the original image, ground truth mask, and predicted mask.
plt.figure(figsize=(14, 8))

# Display the original image.
plt.subplot(231)
plt.title("Original Image")
plt.imshow(test_image)

# Display the original masked image (ground truth).
plt.subplot(232)
plt.title("Original Masked image")
plt.imshow(ground_truth_image)

# Display the predicted segmentation mask.
plt.subplot(233)
plt.title("Predicted Image")
plt.imshow(predicted_image)


In [ ]:
model.save("satellite_segmentation_full.h5")


In [ ]:

# List files and directories in the current directory with human-readable sizes.
!ls -lah
